In [ ]:
## 认证

# https://developers.google.com/earth-engine/guides/python_install
import os
os.environ['HTTP_PROXY'] = 'http://127.0.0.1:10809'
os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:10809'

import ee
ee.Authenticate()
ee.Initialize()

In [ ]:
print(ee.Image("NASA/NASADEM_HGT/001").get("title").getInfo())


In [ ]:
# Define a method for displaying Earth Engine image tiles on a folium map.
def add_ee_layer(self, ee_object, vis_params, name):

    try:
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
                tiles=map_id_dict['tile_fetcher'].url_format,
                attr='Google Earth Engine',
                name=name,
                overlay=True,
                control=True
            ).add_to(self)
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
                tiles=map_id_dict['tile_fetcher'].url_format,
                attr='Google Earth Engine',
                name=name,
                overlay=True,
                control=True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):
            folium.GeoJson(
                data=ee_object.getInfo(),
                name=name,
                overlay=True,
                control=True
            ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
                tiles=map_id_dict['tile_fetcher'].url_format,
                attr='Google Earth Engine',
                name=name,
                overlay=True,
                control=True
            ).add_to(self)

    except:
        print("Could not display {}".format(name))


# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer


In [ ]:
# Add custom basemaps to folium
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles='https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr='Google',
        name='Google Maps',
        overlay=True,
        control=True
    ),
    'Google Satellite': folium.TileLayer(
        tiles='https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr='Google',
        name='Google Satellite',
        overlay=True,
        control=True
    ),
    'Google Terrain': folium.TileLayer(
        tiles='https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr='Google',
        name='Google Terrain',
        overlay=True,
        control=True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles='https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr='Google',
        name='Google Satellite',
        overlay=True,
        control=True
    ),
    'Esri Satellite': folium.TileLayer(
        tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr='Esri',
        name='Esri Satellite',
        overlay=True,
        control=True
    )
}


In [ ]:
# 

In [ ]:
import folium
from folium import plugins


## 全球DEM数据

In [ ]:
dem = ee.Image('USGS/SRTMGL1_003')

# Set visualization parameters.
vis_params = {
  'min': 0,
  'max': 4000,
  'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}

# Create a folium map object.
my_map = folium.Map(location=[20, 0], zoom_start=3, height=500)

# Add custom basemaps
basemaps['Google Maps'].add_to(my_map)
basemaps['Google Satellite Hybrid'].add_to(my_map)

# Add the elevation model to the map object.
my_map.add_ee_layer(dem.updateMask(dem.gt(0)), vis_params, 'DEM')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Add fullscreen button
plugins.Fullscreen().add_to(my_map)

# Display the map.
display(my_map)

## 获取城市中心

In [ ]:
centerName ='Shanghai'
center = ee.FeatureCollection("users/zhouzz400/Boundries/city_center").filter(ee.Filter.eq("NAME",centerName)).geometry()
print(center.getInfo())

x,y = center.getInfo()['coordinates']
Map = folium.Map(location=[y, x], zoom_start=10, height=500)
Map.add_ee_layer(center,{},"center")

display(Map)

## 数据集

In [ ]:
year_dic = {34:1985,33:1986,32:1987,31:1988,
            30:1989,29:1990,28:1991,27:1992,26:1993,25:1994,24:1995,23:1996,22:1997,21:1998,
            20:1999,19:2000,18:2001,17:2002,16:2003,15:2004,14:2005,13:2006,12:2007,11:2008,
            10:2009, 9:2010, 8:2011, 7:2012, 6:2013, 5:2014, 4:2015, 3:2016, 2:2017, 1:2018,} 

In [ ]:
shanghai = ee.Image("users/zhouzz400/demo/shanghai_GAIA")
Viz_GAIA = {"min": 1, "max": 34, "palette": ['FFFFFF', 'FF0000']}
Map.add_ee_layer(shanghai, Viz_GAIA,'gaia')

year= 29
sh_year = shanghai.gte(year)
Viz_year = {"min": 0, "max": 1, "palette": ['FFFFFF', 'FF0000']}
Map.add_ee_layer(sh_year,Viz_year,"sh_year")

water = ee.Image("JRC/GSW1_1/YearlyHistory/"+str(year_dic[year])).gte(2)
Viz_water = {"min": 0, "max": 1, "palette": ["ffffff","0000ff"]}
Map.add_ee_layer(water,Viz_water,"water")

# Add a layer control panel to the map.
Map.add_child(folium.LayerControl())
plugins.Fullscreen().add_to(Map)
# Display the map.
display(Map)


## 计算圈层城市土地密度

In [ ]:
import numpy as np
import pandas as pd
def getBandName(center):
    coor = center.getInfo()["coordinates"]
    print("coordinate: ",coor)
    BandLat = np.round(coor[1],0)
    left = "users/zhouzz400/GAIA_2018_lat/GAIA_1985_2018_"
    if BandLat>0:
        Bandname = left+"%02d"% BandLat
        ceilBandname = left+"%02d"% (BandLat+1)
        floorBandname = left+"%02d"% (BandLat-1)
    elif BandLat<-1:
        Bandname = left+"%03d"% BandLat
        ceilBandname = left+"%03d"% (BandLat+1)
        floorBandname = left+"%03d"% (BandLat-1)
    elif BandLat==0:
        Bandname = left+"%02d"% BandLat
        ceilBandname = left+"%02d"% (BandLat+1)
        floorBandname = left+"%03d"% (BandLat-1)
    elif BandLat==-1:
        Bandname = left+"%03d"% BandLat
        ceilBandname = left+"%02d"% (BandLat+1)
        floorBandname = left+"%03d"% (BandLat-1)
    return Bandname,ceilBandname,floorBandname     

def getDensity(center, years=[34, 24, 14, 4], maxdis=30, kdens=0.1):
    Bandname, ceilBandname, floorBandname = getBandName(center)

    GAIA = ee.ImageCollection([Bandname, ceilBandname, floorBandname]).mosaic()
    #Viz_GAIA = {"min": 1, "max": 34, "palette": ['FFFFFF', 'FF0000']}
    # Map.addLayer(GAIA,Viz_GAIA)

    df = pd.DataFrame(
        columns=["year", "dis", "ring_area", "water_area", "urban_area", "dens"])
    for year in years:
        print("year begin: "+str(year_dic[year]))

        GAIA_year = GAIA.gte(year)
        #Viz_year = {"min": 0, "max": 1, "palette": ['FFFFFF', 'FF0000']}
        #Map.addLayer(sh_year,Viz_year,"sh_year")

        water = ee.Image("JRC/GSW1_1/YearlyHistory/" +
                         str(year_dic[year])).gte(2)
        # Viz_water = {"min": 0, "max": 1, "palette": ["ffffff","0000ff"]}
        # Map.addLayer(water,Viz_water,"water")

        # center = ee.Geometry.Point([121.46851393726786, 31.224416065753665])
        buffer = []
        ring = []

        for i in range(maxdis):
            dis = 1000*(i+1)
            buffer.append(center.buffer(dis))
            if i == 0:
                ring.append(buffer[i])

            else:
                ring.append(buffer[i].symmetricDifference(buffer[i-1]))

            # Map.addLayer(ring[i])
            # gaia_ring = GAIA.clip(ring[i])
            # Map.addLayer(gaia_ring)

            ring_urban = GAIA_year.eq(1).clip(ring[i])
            urban_image = ring_urban.multiply(ee.Image.pixelArea())
            urban_area = ee.Number(urban_image.reduceRegion(
                **{"reducer": ee.Reducer.sum(), "scale": 30, "maxPixels": 1e9}).get("b1"))

            ring_water = water.eq(1).clip(ring[i])
            water_image = ring_water.multiply(ee.Image.pixelArea())
            water_area = ee.Number(water_image.reduceRegion(
                **{"reducer": ee.Reducer.sum(), "scale": 30, "maxPixels": 1e9}).get("waterClass"))

            ring_area = ring[i].area()
            #print(ring_area.getInfo(),water_area.getInfo(),urban_area.getInfo())

            Denominator = ring_area.subtract(water_area)
            dens = urban_area.divide(Denominator).getInfo()
            #print(i,dens)

            dic = {"year": str(year_dic[year]), "dis": dis, "ring_area": ring_area.getInfo(), "water_area": water_area.getInfo(),
                   "urban_area": urban_area.getInfo(), "dens": dens}
            df = df.append(dic, ignore_index=True)

            if dens <= kdens:
                print("less than "+str(kdens)+" in:"+str(i))
                break
    return df


df = getDensity(center, years= [24,4], maxdis=30, kdens=0.4)

## 圈层密度绘图

In [ ]:
dens =  df[df.year=='2015'].dens.values

import matplotlib.pyplot as plt
plt.scatter([i for i in range(len(dens))],dens)

## 城市土地密度比较

<img src="./image/6_China_Shanghai.png" alt="drawing" style="float: left" width="550"/>
<img src="./image/96_Germany_Berlin.png" alt="drawing" style="float: right" width="550"/>